In [2]:
!pip  install  -i  https://pypi.doubanio.com/simple/  --trusted-host pypi.doubanio.com xgboost catboost lightgbm

In [245]:
import warnings
from tqdm import tqdm
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import catboost as cab
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_recall_fscore_support,roc_curve,auc,roc_auc_score
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import gc
#from featexp import get_univariate_plots#用于特征筛选，需要先安装featexp
warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif']=['Simhei']
plt.rcParams['axes.unicode_minus']=False
import json

In [252]:
use_bl_data=True#是否使用baseline的数据处理方式：数值编码:线上0.8537/,one-hot编码:线上0.8530
if use_bl_data:#是否使用baseline的数据处理方式
    print('使用baseline的处理方式:对object进行数值编码....')
    train_df = pd.read_csv('/home/kesci/data/competition_A/train_set.csv').replace(' ', -1)#.fillna(-1)
    train_df.dropna(subset=['肝炎'],inplace=True)
    train_df=train_df.reset_index(drop=True)
    train_df=train_df.fillna(-1)
    train_group = train_df.groupby('护理来源')
    del train_df
    gc.collect()

    dic_1 = {'Private Hospital':1, -1:2, 'Governament Hospital':3, 'Never Counsulted':4, 'clinic':5}
    buf_1 = pd.DataFrame()
    for name,group in train_group:
        group['护理来源'] = dic_1[name]
        buf_1 = pd.concat([buf_1,group],ignore_index=True)

    dic_2 = {'F':1, 'M':2}
    buf_1_group = buf_1.groupby('性别')
    buf_2 = pd.DataFrame()
    for name,group in buf_1_group:
        group['性别'] = dic_2[name]
        buf_2 = pd.concat([buf_2,group],ignore_index=True)
        
    dic_3 = {'north':1, 'east':2, 'south':3, 'west':4}    
    buf_3 = pd.DataFrame()
    buf_2_group = buf_2.groupby('区域')
    for name,group in buf_2_group:
        group['区域'] = dic_3[name]
        buf_3 = pd.concat([buf_3,group],ignore_index=True)

    buf_3 = buf_3.astype(float)

    cat_list = ['肥胖腰围',
                '教育', '未婚',
                '护理来源', '视力不佳',
                '饮酒', '高血压',
                '家庭高血压', '糖尿病',
                '家族糖尿病', '肝炎', '家族肝炎',
                '慢性疲劳']
    for i in cat_list:
        buf_3[i] = buf_3[i].astype(int)

    train_df=buf_3
    #
    test_df = pd.read_csv('/home/kesci/data/competition_A/test_set.csv').replace(' ', -1).fillna(-1)
    test_group = test_df.groupby('护理来源')
    del test_df
    gc.collect()

    dic_1 = {'Private Hospital':1, -1:2, 'Governament Hospital':3, 'Never Counsulted':4, 'clinic':5}
    buf_1 = pd.DataFrame()
    for name,group in test_group:
        group['护理来源'] = dic_1[name]
        buf_1 = pd.concat([buf_1,group],ignore_index=True)

    dic_2 = {'F':1, 'M':2}
    buf_1_group = buf_1.groupby('性别')
    buf_2 = pd.DataFrame()
    for name,group in buf_1_group:
        group['性别'] = dic_2[name]
        buf_2 = pd.concat([buf_2,group],ignore_index=True)
        
    dic_3 = {'north':1, 'east':2, 'south':3, 'west':4}    
    buf_3 = pd.DataFrame()
    buf_2_group = buf_2.groupby('区域')
    for name,group in buf_2_group:
        group['区域'] = dic_3[name]
        buf_3 = pd.concat([buf_3,group],ignore_index=True)

    buf_3 = buf_3.astype(float)

    cat_list = ['肥胖腰围',
                '教育', '未婚',
                '护理来源', '视力不佳',
                '饮酒', '高血压',
                '家庭高血压', '糖尿病',
                '家族糖尿病', '家族肝炎',
                '慢性疲劳']
    for i in cat_list:
        buf_3[i] = buf_3[i].astype(int)
    test_df=buf_3
    #print(train_df.shape,test_df.shape)
    #
    test_data = test_df.drop(columns=['ID'])
    kind = train_df['肝炎']
    train_data = train_df.drop(columns=['ID','肝炎'])
    data_all = train_data.append(test_data)
    #
    #print(train_data.shape,test_data.shape,data_all.shape)
    data_train=data_all[:5984]
    data_test=data_all[5984:]
    print(train_data.shape,test_data.shape)
else:
    print('使用qyl的处理方式:对object进行one-hot编码....')
    train_df=pd.read_csv('/home/kesci/data/competition_A/train_set.csv')
    train_df.dropna(subset=['肝炎'],inplace=True)
    train_df=train_df.reset_index(drop=True)
    #
    test_df=pd.read_csv('/home/kesci/data/competition_A/test_set.csv')
    #
    #将原始数据中所有的列按照float64/object分开
    #合并二者
    all_data=pd.concat((train_df, test_df), axis=0)
    all_data=all_data.drop(['肝炎','ID'],axis=1)
    X_data=all_data
    s=X_data.dtypes
    list_index=s.index.tolist()
    list_obj=[]
    for per in list_index:
        if s[per]=='object':
            list_obj.append(per)
    list_float=list(set(list_index).difference(set(list_obj)))

    float_data=X_data[list_float]#数值形数据
    obj_data=X_data[list_obj]#obj形数据
    #
    #对object列进行one-hot编码
    X_obj=pd.get_dummies(X_data[list_obj])
    X_float=float_data
    #
    X_data_processed = pd.concat((X_obj, X_float), axis=1)
    X_data_processed=X_data_processed.fillna(0)
    X_data_processed.head(2)
    #
    train_data=X_data_processed[:5984]
    test_data=X_data_processed[5984:]
    #
    #
    kind=train_df['肝炎']
    print(train_data.shape,test_data.shape)

使用baseline的处理方式:对object进行数值编码....
(5984, 29) (2785, 29)


In [247]:
def eval_score(y_test,y_pre):
    #fpr, tpr, thresholds = roc_curve(y_test, y_pre)
    #auc(fpr, tpr)
    return roc_auc_score(y_test, y_pre[:,1])
#
def k_fold_serachParmaters(model,train_data,kind):
    mean_auc=0
    mean_aucTrain=0
    n_splits=5
    sk = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2020)
    for train, test in sk.split(train_data, kind):
        x_train = train_data.iloc[train]
        y_train = kind.iloc[train]
        x_test = train_data.iloc[test]
        y_test = kind.iloc[test]

        model.fit(x_train, y_train)
        pred = model.predict_proba(x_test)
        per_auc =  eval_score(y_test,pred)
        mean_auc+=per_auc/n_splits
        
        pred_Train = model.predict_proba(x_train)
        per_auc_train =  eval_score(y_train,pred_Train)
        mean_aucTrain+=per_auc_train/n_splits
    print('mean valAuc:',mean_auc)
    print('mean trainAuc:',mean_aucTrain)

In [206]:
xlf=xgb.XGBClassifier(max_depth=3
                      ,learning_rate=0.08
                      ,n_estimators=100
                      ,reg_alpha=0.004
                      ,n_jobs=8
                      ,importance_type='total_cover'
                     )
k_fold_serachParmaters(xlf,train_data,kind)

mean valAuc: 0.8679935303465112
mean trainAuc: 0.9351726741157957


In [200]:
llf=lgb.LGBMClassifier(num_leaves=3
                           ,max_depth=4
                           ,learning_rate=0.1
                           ,n_estimators=150
                           ,n_jobs=8
                           ,reg_alpha=0
                           ,reg_lambda=0
                           )
k_fold_serachParmaters(llf,train_data,kind)                 

mean valAuc: 0.8699330448265767
mean trainAuc: 0.9186626477681271


In [212]:
clf=cab.CatBoostClassifier(iterations=150
                              ,learning_rate=0.05
                              ,depth=5
                              ,silent=True
                              ,thread_count=8
                              ,task_type='CPU'
                              )
k_fold_serachParmaters(clf,train_data,kind)

mean valAuc: 0.874445873028861
mean trainAuc: 0.9234131890961794


In [226]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import learning_curve, GridSearchCV
rf = RandomForestClassifier(oob_score=True, random_state=2020,
            n_estimators= 120,max_depth=11,min_samples_split=170)
k_fold_serachParmaters(rf,train_data,kind)

mean valAuc: 0.8673595361252031
mean trainAuc: 0.92358183816173


In [250]:
#features=important_frt[:15]
#train_data=train_data[features]
#test_data=test_data[features]

In [254]:
#
details = []
answers = []
mean_auc=0
n_splits=10
sk = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2018)
cnt=0
for train, test in sk.split(train_data, kind):
    x_train = train_data.iloc[train]
    y_train = kind.iloc[train]
    x_test = train_data.iloc[test]
    y_test = kind.iloc[test]

    xlf.fit(x_train, y_train)
    pred_xgb = xlf.predict_proba(x_test)
    weight_xgb = eval_score(y_test,pred_xgb)

    llf.fit(x_train, y_train)
    pred_llf = llf.predict_proba(x_test)
    weight_lgb = eval_score(y_test,pred_llf)

    clf.fit(x_train, y_train)
    pred_cab = clf.predict_proba(x_test)
    weight_cab =  eval_score(y_test,pred_cab)

    rf.fit(x_train, y_train)
    pred_rf = rf.predict_proba(x_test)
    weight_rf =  eval_score(y_test,pred_rf)


    prob_xgb = xlf.predict_proba(x_test)
    prob_lgb = llf.predict_proba(x_test)
    prob_cab = clf.predict_proba(x_test)
    prob_rf = rf.predict_proba(x_test)

    scores = []
    ijkl = []
    weight = np.arange(0, 1.05, 0.1)
    for i, item1 in enumerate(weight):
        for j, item2 in enumerate(weight[weight <= (1 - item1)]):
            for k, item3 in enumerate(weight[weight <= (1 - item1-item2)]):
                prob_end = prob_xgb * item1 + prob_lgb * item2 + prob_cab *item3+prob_rf*(1 - item1 - item2-item3)
                score = eval_score(y_test,prob_end)
                scores.append(score)
                ijkl.append((item1, item2,item3, 1 - item1 - item2-item3))

    ii = ijkl[np.argmax(scores)][0]
    jj = ijkl[np.argmax(scores)][1]
    kk = ijkl[np.argmax(scores)][2]
    ll = ijkl[np.argmax(scores)][3]

    details.append(max(scores))
    details.append(weight_xgb)
    details.append(weight_lgb)
    details.append(weight_cab)
    details.append(weight_rf)
    details.append(ii)
    details.append(jj)
    details.append(kk)
    details.append(ll)

    cnt+=1
    print('每{}次验证的auc:{}'.format(cnt,max(scores)))
    mean_auc+=max(scores)/n_splits

    test_xgb = xlf.predict_proba(test_data)
    test_lgb = llf.predict_proba(test_data)
    test_cab = clf.predict_proba(test_data)
    test_rf = rf.predict_proba(test_data)
    #加权平均
    #ans = test_xgb * ii + test_lgb * jj + test_cab * kk + test_rf*ll#加权平均
    #加权几何平均
    ans = np.sqrt(test_xgb**2 * ii + test_lgb**2 * jj + test_cab**2 * kk + test_rf**2*ll)

    answers.append(ans)
print('mean weighted auc:',mean_auc)

每1次验证的auc:0.9244731445525546
每2次验证的auc:0.8685806536061783
每3次验证的auc:0.8735663082437276
每4次验证的auc:0.8757168458781361
每5次验证的auc:0.8711917562724014
每6次验证的auc:0.8887096774193548
每7次验证的auc:0.8989695340501793
每8次验证的auc:0.8648297491039426
每9次验证的auc:0.8692204301075268
每10次验证的auc:0.8927867383512544
mean weighted auc: 0.8828044837585256


## 实验记录
`xlf=xgb.XGBClassifier(max_depth=3
                      ,learning_rate=0.08
                      ,n_estimators=100
                      ,reg_alpha=0.004
                      ,n_jobs=8
                      ,importance_type='total_cover'
                     )
llf=lgb.LGBMClassifier(num_leaves=3
                           ,max_depth=4
                           ,learning_rate=0.1
                           ,n_estimators=150
                           ,n_jobs=8
                           ,reg_alpha=0
                           ,reg_lambda=0
                           )
clf=cab.CatBoostClassifier(iterations=150
                              ,learning_rate=0.05
                              ,depth=5
                              ,silent=True
                              ,thread_count=8
                              ,task_type='CPU'
                              )
rf = RandomForestClassifier(oob_score=True, random_state=2020,
            n_estimators= 120,max_depth=11,min_samples_split=170)
													 `
**十折交叉验证**：mean weighted auc: 0.8828 随机种子:2018
**线上:** 8548

## 观察训练/验证过程

In [230]:
df=pd.DataFrame(np.array(details).reshape(int(len(details)/9),9)
                ,columns=['test_end_score','xgboost','lightgbm','catboost','rf'
                ,'weight_xgboost','weight_lightgbm','weight_catboost','weight_rf'])
df

,test_end_score,xgboost,lightgbm,catboost,rf,weight_xgboost,weight_lightgbm,weight_catboost,weight_rf
0,0.924473,0.909987,0.924473,0.914132,0.906322,0.0,1.0,0.0,0.0
1,0.868581,0.852960,0.858458,0.868581,0.845805,0.0,0.0,1.0,0.0
2,0.873566,0.842115,0.848970,0.838396,0.872312,0.0,0.2,0.0,0.8
3,0.875717,0.864337,0.866622,0.875179,0.857751,0.0,0.1,0.9,0.0
4,0.871192,0.850090,0.842339,0.855556,0.870878,0.0,0.0,0.3,0.7
5,0.888710,0.882975,0.880421,0.874955,0.885977,0.4,0.0,0.0,0.6
6,0.898970,0.888441,0.886514,0.890860,0.898477,0.1,0.0,0.0,0.9
7,0.864830,0.848342,0.857885,0.864651,0.844400,0.0,0.1,0.9,0.0
8,0.869220,0.866263,0.868683,0.857930,0.832124,0.2,0.7,0.0,0.1
9,0.892787,0.885753,0.886604,0.880376,0.880197,0.0,0.7,0.0,0.3


In [231]:
df.mean()

test_end_score     0.882804
xgboost            0.869126
lightgbm           0.872097
catboost           0.872062
rf                 0.869424
weight_xgboost     0.070000
weight_lightgbm    0.280000
weight_catboost    0.310000
weight_rf          0.340000
dtype: float64

In [257]:
#观察训练/验证过程
df=pd.DataFrame(np.array(details).reshape(int(len(details)/9),9)
                ,columns=['test_end_score','xgboost','lightgbm','catboost','rf'
                ,'weight_xgboost','weight_lightgbm','weight_catboost','weight_rf'])
df

df.mean()

xlf_impt=xlf.feature_importances_
llf_impt=llf.feature_importances_/sum(llf.feature_importances_)
clf_impt=clf.feature_importances_/sum(clf.feature_importances_)

importance=pd.DataFrame({
    'column':train_data.columns,
    'importance':xlf_impt+llf_impt+clf_impt,
}).sort_values(by='importance')
importance=importance.reset_index(drop=True)
important_frt=list(importance['column'].values)
important_frt.reverse()
#print(important_frt)
#这里解码一下特征名字
#important_frt_decode=[frt_name[name] for name in important_frt]

print(important_frt)#打印最重要的前30个特征

#fina=sum(answers)/n_splits#
fina=np.sqrt(sum(np.array(answers)**2)/n_splits)#几何平均
fina=fina[:,1]
test_df['hepatitis']=fina#可选:fina_persudo是伪标签的预测结果
submit_csv=test_df[['ID','hepatitis']]
submit_csv.to_csv('submit10_14Geom_Geom.csv',index=False)

['年龄', '慢性疲劳', '好胆固醇', '最低血压', '最高血压', '高血压', '腰围', '体重指数', '坏胆固醇', '性别', '体育活动', '区域', '总胆固醇', '身高', '收入', '体重', '家庭高血压', '视力不佳', 'ALF', '教育', 'PVD', '未婚', '肥胖腰围', '饮酒', '家族糖尿病', '糖尿病', '护理来源', '血脂异常', '家族肝炎']


In [256]:
submit_csv.describe()

,ID,hepatitis
count,2785.000000,2785.000000
mean,4455.957989,0.066817
std,2530.334154,0.089623
min,1.000000,0.001552
25%,2326.000000,0.009745
50%,4470.000000,0.030922
75%,6659.000000,0.094694
max,8776.000000,0.699707


In [259]:
submit_csv.head(5)

,ID,hepatitis
0,5258.0,0.170342
1,1109.0,0.003415
2,1522.0,0.325484
3,8011.0,0.128887
4,5301.0,0.018667


In [258]:
!./kesci_submit -token b4c08594d83377af -file /home/kesci/work/submit10_14Geom_Geom.csv

Kesci Submit Tool 4.0.0

> 已验证Token
> 提交文件 /home/kesci/work/submit10_14Geom_Geom.csv (74.16 KiB), Target Qiniu
> 已上传 100 %
> 文件已上传        
> 服务器响应: 200 提交成功，请等待评审完成
> 提交完成


In [238]:

'''
answers=[]
s1=pd.read_csv('submit10_14Geom_8537.csv').sort_values(['ID']).reset_index(drop=True)
answers.append(s1['hepatitis'].values)
s2=pd.read_csv('submit10_14Geom_8530.csv').sort_values(['ID']).reset_index(drop=True)
answers.append(s2['hepatitis'].values)
answers=np.array(answers)
fina=np.sqrt(sum(np.array(answers)**2)/answers.shape[0])#几何平均
test_df=test_df.sort_values(['ID']).reset_index(drop=True)
test_df['hepatitis']=fina#可选:fina_persudo是伪标签的预测结果
submit_csv=test_df[['ID','hepatitis']]
#submit_csv.to_csv('submit10_14Geom_8541.csv',index=False)
'''